#  任务4 模型选择(3天)

# 以lightGBM为例

In [21]:
from __future__ import print_function
import lightgbm as lgb
import sklearn
import numpy
import hyperopt
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
import colorama
import numpy as np
from sklearn.model_selection import KFold,StratifiedKFold
import pandas as pd
from sklearn.metrics import r2_score

X_train = pd.read_csv('./Filter_train_data.csv')
print(X_train.columns)
Y_train = pd.read_csv('./train_data.csv').pop("tradeMoney")
N_HYPEROPT_PROBES = 500
HYPEROPT_ALGO = tpe.suggest  #  tpe.suggest OR hyperopt.rand.suggest

# ----------------------------------------------------------
# 在控制台、命令行输出彩色文字的模块
colorama.init()

# ---------------------------------------------------------------------
# 设置参数
def get_lgb_params(space):
    lgb_params = dict()
    lgb_params['boosting_type'] = space['boosting_type'] if 'boosting_type' in space else 'gbdt'  # 未指定使用梯度提升决策树
    lgb_params['objective'] = 'regression'  # L2回归类型
    lgb_params['metric'] = 'rmse'  # 评估指标
    lgb_params['learning_rate'] = space['learning_rate']
    lgb_params['num_leaves'] = int(space['num_leaves'])   # 叶子数
    lgb_params['min_data_in_leaf'] = int(space['min_data_in_leaf'])  # 一个叶子上的最小数据量，防止生长为较深的树
    lgb_params['min_sum_hessian_in_leaf'] = space['min_sum_hessian_in_leaf']  # 一个叶子的最小海塞矩阵之和
    lgb_params['max_depth'] = -1  # 不指定最大深度
    lgb_params['lambda_l1'] = space['lambda_l1'] if 'lambda_l1' in space else 0.0   # 正则项
    lgb_params['lambda_l2'] = space['lambda_l2'] if 'lambda_l2' in space else 0.0
    lgb_params['max_bin'] = int(space['max_bin']) if 'max_bin' in space else 256 #   存储特征的最大箱数
    lgb_params['feature_fraction'] = space['feature_fraction']  # 随机抽取特征的比例
    lgb_params['bagging_fraction'] = space['bagging_fraction']  # 在不进行重采样的情况下随机选择部分数据的比例
    lgb_params['bagging_freq'] = int(space['bagging_freq']) if 'bagging_freq' in space else 1   # bagging的频率，k意味着每k次迭代执行bagging
    lgb_params['nthread'] = 4  #线程数
    return lgb_params

# ---------------------------------------------------------------------

obj_call_count = 0  # 计算周期
cur_best_score = 0 # 0 or np.inf
log_writer = open( '../lgb-hyperopt-log.txt', 'w' )


def objective(space):
    global obj_call_count, cur_best_score

    obj_call_count += 1

    print('\nLightGBM objective call #{} cur_best_score={:7.5f}'.format(obj_call_count,cur_best_score) )

    lgb_params = get_lgb_params(space)  # 读取参数

    sorted_params = sorted(space.items(), key=lambda z: z[0])
    params_str = str.join(' ', ['{}={}'.format(k, v) for k, v in sorted_params])
    print('Params: {}'.format(params_str) )
    
    kf = KFold(n_splits=3, shuffle=True, random_state=0)
    out_of_fold = np.zeros(len(X_train))
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
        D_train = lgb.Dataset(X_train.iloc[train_idx], label=Y_train[train_idx])
        D_val = lgb.Dataset(X_train.iloc[val_idx], label=Y_train[val_idx])
        # Train
        num_round = 10000
        clf = lgb.train(lgb_params,
                           D_train,
                           num_boost_round=num_round,
                           # metrics='mlogloss',
                           valid_sets=D_val,
                           # valid_names='val',
                           # fobj=None,
                           # feval=None,
                           # init_model=None,
                           # feature_name='auto',
                           # categorical_feature='auto',
                           early_stopping_rounds=200,
                           # evals_result=None,
                           verbose_eval=False,
                           # learning_rates=None,
                           # keep_training_booster=False,
                           # callbacks=None
                           )
        # predict
        nb_trees = clf.best_iteration  # 选择指定模型的最佳迭代
        val_loss = clf.best_score['valid_0']
        print('nb_trees={} val_loss={}'.format(nb_trees, val_loss))
        out_of_fold[val_idx] = clf.predict(X_train.iloc[val_idx], num_iteration=nb_trees)
        score = r2_score(out_of_fold, Y_train)

    print('val_r2_score={}'.format(score))

    log_writer.write('score={} Params:{} nb_trees={}\n'.format(score, params_str, nb_trees ))
    log_writer.flush()

    if score>cur_best_score:
        cur_best_score = score
        print(colorama.Fore.GREEN + 'NEW BEST SCORE={}'.format(cur_best_score) + colorama.Fore.RESET)
    return {'loss': -score, 'status': STATUS_OK}

# --------------------------------------------------------------------------------
# 搜索空间
space ={
        'num_leaves': hp.quniform ('num_leaves', 10, 100, 1),
        'min_data_in_leaf':  hp.quniform ('min_data_in_leaf', 10, 100, 1),
        'feature_fraction': hp.uniform('feature_fraction', 0.75, 1.0),
        'bagging_fraction': hp.uniform('bagging_fraction', 0.75, 1.0),
        'learning_rate': hp.uniform('learning_rate', 0, 0.01),
#         'learning_rate': hp.loguniform('learning_rate', -5.0, -2.3),
        'min_sum_hessian_in_leaf': hp.loguniform('min_sum_hessian_in_leaf', 0, 2.3),
        'max_bin': hp.quniform ('max_bin', 88, 200, 1),
        'bagging_freq': hp.quniform ('bagging_freq', 1, 15, 1),
        'lambda_l1': hp.uniform('lambda_l1', 0, 10 ),
        'lambda_l2': hp.uniform('lambda_l2', 0, 10 ),
       }

trials = Trials()  # 查看黑匣子的参数
best = hyperopt.fmin(fn=objective,
                     space=space,
                     algo=HYPEROPT_ALGO,
                     max_evals=N_HYPEROPT_PROBES,
                     trials=trials,
                     verbose=1)

print('-'*50)
print('The best params:')
print( best )
print('\n\n')

Index(['Unnamed: 0', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       '140', '141', '142', '143', '144', '145', '146', '147', '148', '149'],
      dtype='object', length=151)
                                                                                                                       
LightGBM objective call #1 cur_best_score=0.00000
Params: bagging_fraction=0.8487949260703118 bagging_freq=9.0 feature_fraction=0.9713006934416635 lambda_l1=0.6313158907789107 lambda_l2=9.817075069470333 learning_rate=0.0014971596249927221 max_bin=91.0 min_data_in_leaf=93.0 min_sum_hessian_in_leaf=4.469713389624021 num_leaves=56.0
nb_trees=2 val_loss=OrderedDict([('rmse', 425403.24040378025)])                                                        
nb_trees=153 val_loss=OrderedDict([('rmse', 855070.864537195)])                                                        
nb_trees=7 val_loss=OrderedDict([('rmse', 10942.453905887725)])                                                       

nb_trees=1 val_loss=OrderedDict([('rmse', 425403.2825240898)])                                                         
nb_trees=246 val_loss=OrderedDict([('rmse', 855058.8651131351)])                                                       
nb_trees=1 val_loss=OrderedDict([('rmse', 10932.901616098588)])                                                        
val_r2_score=-8286.960256392806                                                                                        
                                                                                                                       
LightGBM objective call #10 cur_best_score=0.00000
Params: bagging_fraction=0.8070654969891983 bagging_freq=13.0 feature_fraction=0.9875821472328721 lambda_l1=4.13752311190994 lambda_l2=6.38441213307467 learning_rate=0.00873123012974931 max_bin=169.0 min_data_in_leaf=42.0 min_sum_hessian_in_leaf=1.6333556620096024 num_leaves=73.0
nb_trees=13 val_loss=OrderedDict([('rmse', 425401.30715133646)])    

Params: bagging_fraction=0.9418207781740117 bagging_freq=2.0 feature_fraction=0.9803848597482079 lambda_l1=1.7709641239630858 lambda_l2=2.3758452404377883 learning_rate=0.004182259430077995 max_bin=188.0 min_data_in_leaf=67.0 min_sum_hessian_in_leaf=4.606982121662296 num_leaves=11.0
nb_trees=1 val_loss=OrderedDict([('rmse', 425403.2406446306)])                                                         
nb_trees=13 val_loss=OrderedDict([('rmse', 855076.2209010803)])                                                        
nb_trees=2 val_loss=OrderedDict([('rmse', 10937.407103878455)])                                                        
val_r2_score=-80376.48055964996                                                                                        
                                                                                                                       
LightGBM objective call #19 cur_best_score=0.00000
Params: bagging_fraction=0.8262258987839666 bagging_freq=15.0 fea

val_r2_score=-66467.79412728401                                                                                        
                                                                                                                       
LightGBM objective call #27 cur_best_score=0.00000
Params: bagging_fraction=0.9947606176671804 bagging_freq=8.0 feature_fraction=0.9503125847674366 lambda_l1=7.287247410275426 lambda_l2=1.288126963870357 learning_rate=0.004114486808413503 max_bin=179.0 min_data_in_leaf=76.0 min_sum_hessian_in_leaf=4.073294814825215 num_leaves=79.0
nb_trees=2 val_loss=OrderedDict([('rmse', 425403.10762999096)])                                                        
nb_trees=17 val_loss=OrderedDict([('rmse', 855075.0898192243)])                                                        
nb_trees=3 val_loss=OrderedDict([('rmse', 10920.458631727746)])                                                        
val_r2_score=-73673.9721056651                                     

nb_trees=17 val_loss=OrderedDict([('rmse', 855071.7176559775)])                                                        
nb_trees=1 val_loss=OrderedDict([('rmse', 10928.245531312392)])                                                        
val_r2_score=-52204.47105513689                                                                                        
                                                                                                                       
LightGBM objective call #36 cur_best_score=0.00000
Params: bagging_fraction=0.8064986320606317 bagging_freq=9.0 feature_fraction=0.966033283254594 lambda_l1=5.979872347814678 lambda_l2=0.9063908712064359 learning_rate=0.001175371057791722 max_bin=114.0 min_data_in_leaf=41.0 min_sum_hessian_in_leaf=2.531220214770293 num_leaves=49.0
nb_trees=11 val_loss=OrderedDict([('rmse', 425402.9371925126)])                                                        
nb_trees=108 val_loss=OrderedDict([('rmse', 855073.2956041408)])   

...

Params: bagging_fraction=0.8187045017738779 bagging_freq=7.0 feature_fraction=0.9215316443943944 lambda_l1=6.056411433625164 lambda_l2=8.621563964512939 learning_rate=0.0048254507097093835 max_bin=138.0 min_data_in_leaf=91.0 min_sum_hessian_in_leaf=2.7231683160301956 num_leaves=27.0
nb_trees=8 val_loss=OrderedDict([('rmse', 425402.6773246004)])                                                         
nb_trees=28 val_loss=OrderedDict([('rmse', 855074.358267166)])                                                         
nb_trees=1 val_loss=OrderedDict([('rmse', 10945.982481185745)])                                                        
val_r2_score=-55993.107018858835                                                                                       
                                                                                                                       
LightGBM objective call #461 cur_best_score=0.00000
Params: bagging_fraction=0.8362714303190758 bagging_freq=9.0 fea

val_r2_score=-7186.952929545043                                                                                        
                                                                                                                       
LightGBM objective call #469 cur_best_score=0.00000
Params: bagging_fraction=0.840963427938235 bagging_freq=6.0 feature_fraction=0.8422966391450419 lambda_l1=4.449499455618822 lambda_l2=8.186457639387763 learning_rate=0.006442186584063149 max_bin=156.0 min_data_in_leaf=79.0 min_sum_hessian_in_leaf=2.424838822993501 num_leaves=20.0
nb_trees=1 val_loss=OrderedDict([('rmse', 425403.24409670907)])                                                        
nb_trees=14 val_loss=OrderedDict([('rmse', 855072.2457551329)])                                                        
nb_trees=1 val_loss=OrderedDict([('rmse', 10943.324267798542)])                                                        
val_r2_score=-67596.40258587465                                    

nb_trees=84 val_loss=OrderedDict([('rmse', 855071.0026545834)])                                                        
nb_trees=1 val_loss=OrderedDict([('rmse', 10934.403862412482)])                                                        
val_r2_score=-18395.586970236363                                                                                       
                                                                                                                       
LightGBM objective call #478 cur_best_score=0.00000
Params: bagging_fraction=0.7752989219716512 bagging_freq=8.0 feature_fraction=0.8807623521406072 lambda_l1=6.627955332841207 lambda_l2=8.806869346234613 learning_rate=0.0069813993702353225 max_bin=157.0 min_data_in_leaf=77.0 min_sum_hessian_in_leaf=1.8958001154392994 num_leaves=16.0
nb_trees=24 val_loss=OrderedDict([('rmse', 425399.7308485876)])                                                        
nb_trees=10 val_loss=OrderedDict([('rmse', 855076.4269190086)]) 

Params: bagging_fraction=0.7839081694423439 bagging_freq=5.0 feature_fraction=0.8943560896102289 lambda_l1=6.711049087951964 lambda_l2=6.119446666837454 learning_rate=0.006322001641792104 max_bin=159.0 min_data_in_leaf=58.0 min_sum_hessian_in_leaf=3.5773448599049424 num_leaves=14.0
nb_trees=5 val_loss=OrderedDict([('rmse', 425402.70595795254)])                                                        
nb_trees=22 val_loss=OrderedDict([('rmse', 855069.5424849407)])                                                        
nb_trees=1 val_loss=OrderedDict([('rmse', 10930.595812478618)])                                                        
val_r2_score=-71519.2202455009                                                                                         
                                                                                                                       
LightGBM objective call #487 cur_best_score=0.00000
Params: bagging_fraction=0.8224670354117833 bagging_freq=6.0 feat

val_r2_score=-39153.9588428728                                                                                         
                                                                                                                       
LightGBM objective call #495 cur_best_score=0.00000
Params: bagging_fraction=0.8291483220797036 bagging_freq=7.0 feature_fraction=0.8924113328599476 lambda_l1=7.6155367011314254 lambda_l2=6.381512942217559 learning_rate=0.005117517908239553 max_bin=135.0 min_data_in_leaf=86.0 min_sum_hessian_in_leaf=2.6050512758715714 num_leaves=19.0
nb_trees=1 val_loss=OrderedDict([('rmse', 425403.28196716256)])                                                        
nb_trees=84 val_loss=OrderedDict([('rmse', 855074.6349599492)])                                                        
nb_trees=1 val_loss=OrderedDict([('rmse', 10947.364388429329)])                                                        
val_r2_score=-17731.51726697985                                 

In [22]:
X=np.array([
    [1,2,3,4],
    [11,12,13,14],
    [21,22,23,24],
    [31,32,33,34],
    [41,42,43,44],
    [51,52,53,54],
    [61,62,63,64],
    [71,72,73,74]
])

y=np.array([1,1,0,0,1,1,0,0])
#n_folds这个参数没有，引入的包不同，
floder = KFold(n_splits=4,random_state=0,shuffle=False)
sfolder = StratifiedKFold(n_splits=4,random_state=0,shuffle=False)
print(sfolder.split(X,y))
for train, test in sfolder.split(X,y):
    print('Train: %s | test: %s' % (train, test))
    print("")
print("KKK")
for train, test in floder.split(X,y):
    print('Train: %s | test: %s' % (train, test))
    print(" ")

D:\Anaconda\envs\mcl\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


<generator object _BaseKFold.split at 0x000001AE32743BA0>
Train: [1 3 4 5 6 7] | test: [0 2]

Train: [0 2 4 5 6 7] | test: [1 3]

Train: [0 1 2 3 5 7] | test: [4 6]

Train: [0 1 2 3 4 6] | test: [5 7]

KKK
Train: [2 3 4 5 6 7] | test: [0 1]
 
Train: [0 1 4 5 6 7] | test: [2 3]
 
Train: [0 1 2 3 6 7] | test: [4 5]
 
Train: [0 1 2 3 4 5] | test: [6 7]
 
